In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3
from tqdm.autonotebook import tqdm

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

C:\Users\Andres\AppData\Local\Temp\ipykernel_285244\1102906597.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Read file with largest interval

In [3]:
db = sqlite3.connect('data\SWAMIS\\100_frames\swamis_05760s_rotAssoc.sdb')
# db = sqlite3.connect('data\SWAMIS\swamisMDI.sdb')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# tabs = pd.read_sql_query('SELECT * FROM tabs WHERE frame = 3173',db)
tabs = pd.read_sql_query('SELECT * FROM tabs',db)

[('params',), ('files',), ('tabs',), ('hist',), ('events',), ('event_hist',)]


In [4]:
list(tabs)

['id',
 'frame',
 'scale',
 'assoc_larger',
 'flux',
 'maxB',
 'str_pix',
 'size',
 'x',
 'y',
 'xerr',
 'yerr',
 'closeness',
 'est_dflux',
 'neighbors',
 'xs',
 'ys',
 'lon',
 'lat',
 'size_s']

### Clean elements that have a larger associated element

In [5]:
# Find elements that have an associated larger element
tabs_assLrg = tabs[np.isfinite(tabs['assoc_larger'])].copy().reset_index(drop=True)

# Find maks of all objects with an associated larger element in at least one frame
msk_assLrg = np.isin(tabs['id'],tabs_assLrg['id'])

# Create table with elements without an associated larger element
tabs_NoassLrg = tabs[np.logical_not(msk_assLrg)].copy().reset_index(drop=True)
tabs_NoassLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,1,1,1,NaN,-409.852418,-77.336472,0,2,2168.542486,3816.000000,0.498192,0.000043,0,-409.852418,None,-66.075702,-891.520053,-14.070494,-73.466620,0.508633
1,2,1,1,NaN,-635.447272,-82.187646,0,3,2046.357913,3811.291415,0.338977,0.321320,1,-635.447272,"-1209,-1207",-4.455403,-889.221020,-0.923948,-73.185057,0.762949
2,3,1,1,NaN,-401.259592,-74.707427,0,2,2042.486745,3810.000000,0.499824,0.000086,1,-401.259592,-1207,-2.502384,-888.572155,-0.515724,-73.077809,0.508633
3,4,1,1,NaN,-744.679039,-95.060790,1,3,1827.657345,3809.082584,0.335591,0.570318,1,-744.679039,-1211,105.836138,-888.242242,21.967109,-72.768028,0.762949
4,5,1,1,NaN,-412.025671,-77.163422,0,2,1877.470658,3809.470658,0.499138,0.499138,1,-412.025671,"-1210,10111",80.715163,-888.407168,16.705742,-72.902785,0.508633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179629,997599,97,8,NaN,137.286775,25.308998,0,2,57.506810,349.000000,0.499954,0.000007,0,NaN,None,-803.933969,378.158168,-65.221185,21.764876,32.633030
1179630,997600,97,8,NaN,64.551843,33.180939,0,2,279.578619,361.000000,0.493780,0.000000,1,NaN,928083,93.096371,426.630616,6.056482,22.600069,32.633030
1179631,997601,97,8,NaN,124.545024,25.215373,0,2,88.508721,392.000000,0.499924,0.000000,1,NaN,988017,-678.705760,551.851104,-58.456934,33.369427,32.633030
1179632,997602,97,8,NaN,235.349490,31.482278,1,3,423.700908,399.688100,0.323756,0.327581,0,NaN,None,675.259708,582.906191,60.914757,35.856594,48.949545


### Calculate lifetimes

In [6]:
# Add life column
tabs_NoassLrg['Life'] = 0

# Calculate lifes
unique_id = np.unique(tabs_NoassLrg['id'])
for i in tqdm(np.arange(0, unique_id.shape[0])):
# for i in np.arange(0,5):
    snglFtr = tabs_NoassLrg[tabs_NoassLrg['id'] == unique_id[i]].copy().reset_index(drop=True)
    tabs_NoassLrg.loc[tabs_NoassLrg['id'] == unique_id[i],'Life'] = (np.max(snglFtr['frame'])-np.min(snglFtr['frame'])+1)



100%|██████████| 745954/745954 [2:22:33<00:00, 87.21it/s]   


### Drop single frame objects and change frame to time

In [7]:
tabs_NoassLrg.to_pickle('data\SWAMIS\\100_frames\swamis_NoAssociate_05760s.pnds')